In [5]:
from pyzbar.pyzbar import decode
from PIL import Image

# Đọc file QR code (đổi đường dẫn nếu khác)
img = Image.open("share2.png")

# Giải mã QR
data = decode(img)

# In nội dung
for obj in data:
    print("Type:", obj.type)
    print("Data:", obj.data.decode("utf-8"))
    
cipher = "71%779<<5=;5;<<799=55<>6;7696:959;888>8;<:"

def caesar_printable(s, shift):
    result = ""
    for c in s:
        if 32 <= ord(c) <= 126:
            result += chr((ord(c) - 32 - shift) % 95 + 32)
        else:
            result += c
    return result

for shift in range(1, 95):
    decoded = caesar_printable(cipher, shift)
    if decoded.startswith("2,") or decoded.startswith("share") or decoded.startswith("x,"):
        print(f"Shift {shift}: {decoded}")




Type: QRCODE
Data: 71%779<<5=;5;<<799=55<>6;7696:959;888>8;<:
Type: QRCODE
Data: Caesar cipher
Shift 5: 2, 224770860677244800791621415404633393675


In [18]:
#!/usr/bin/env python3
# save as try_pdf_hint_secret.py
# Requirements: pip install pypdf pycryptodome

import re, binascii, sys
from itertools import cycle
from pathlib import Path
from typing import Optional
from pypdf import PdfReader
from Crypto.Cipher import AES, ARC4
from Crypto.Hash import MD5, SHA256

# ---------- helpers ----------
def find_hex_fields(pdf_path: str):
    b = Path(pdf_path).read_bytes()
    # non-greedy patterns
    hint_m = re.search(rb"/Hint\s*<([0-9A-Fa-f]+)>", b)
    secret_m = re.search(rb"/Secret\s*<([0-9A-Fa-f]+)>", b)
    o_m = re.search(rb"/O\s*<([0-9A-Fa-f]+)>", b)
    u_m = re.search(rb"/U\s*<([0-9A-Fa-f]+)>", b)
    return {
        "hint": hint_m.group(1).decode() if hint_m else None,
        "secret": secret_m.group(1).decode() if secret_m else None,
        "O": o_m.group(1).decode() if o_m else None,
        "U": u_m.group(1).decode() if u_m else None
    }

def is_mostly_printable(bs: bytes, threshold=0.8):
    if not bs: return False
    good = sum(32 <= b < 127 for b in bs)
    return (good / len(bs)) >= threshold

def try_pdf_decrypt(pdf_path: str, password: str) -> bool:
    try:
        r = PdfReader(pdf_path)
        res = r.decrypt(password)
        # pypdf returns 0 if wrong, 1/2 for success in some versions; accept truthy
        if res:
            print(f"[+] SUCCESS decrypt with password: {password}")
            try:
                print("Metadata:", r.metadata)
            except Exception:
                print("Metadata access might be restricted but decrypt succeeded.")
            return True
        else:
            return False
    except Exception as e:
        # some versions throw different errors
        print("decrypt call error:", e)
        return False

# ---------- decode attempts ----------
def attempt_xor(secret_bytes: bytes, hint_bytes: bytes):
    print("[*] Trying repeating-key XOR with hint...")
    out = bytes(a ^ b for a, b in zip(secret_bytes, cycle(hint_bytes)))
    print("-> XOR result sample (first 200 bytes):")
    print(out[:200])
    if is_mostly_printable(out[:200]) or b"%PDF" in out or b"flag" in out or b"Password" in out:
        print("[!] XOR likely successful or interesting. Dumping more:")
        print(out[:2000])
        return out
    return None

def derive_keys_from_hint(hint_bytes: bytes):
    # produce candidate keys of 16/24/32 bytes
    md5 = MD5.new(hint_bytes).digest()
    sha256 = SHA256.new(hint_bytes).digest()
    # raw repeated/padded
    raw16 = (hint_bytes * ((16 + len(hint_bytes) - 1)//len(hint_bytes)))[:16]
    raw32 = (hint_bytes * ((32 + len(hint_bytes) - 1)//len(hint_bytes)))[:32]
    cand = {
        "raw16": raw16,
        "raw32": raw32,
        "md5": md5,
        "sha256_16": sha256[:16],
        "sha256_32": sha256
    }
    return cand

def try_aes_variants(secret_bytes: bytes, hint_bytes: bytes):
    print("[*] Trying AES variants (ECB/CBC) with keys derived from hint...")
    keys = derive_keys_from_hint(hint_bytes)
    results = {}
    for name, key in keys.items():
        for mode_name in ("ECB","CBC"):
            try:
                if mode_name == "ECB":
                    cipher = AES.new(key, AES.MODE_ECB)
                    pt = cipher.decrypt(secret_bytes)
                else:
                    # use first 16 bytes of hint (or zeros) as IV
                    iv = (hint_bytes * 2)[:16] if len(hint_bytes) >= 16 else (hint_bytes.ljust(16, b'\x00'))
                    cipher = AES.new(key, AES.MODE_CBC, iv=iv)
                    pt = cipher.decrypt(secret_bytes)
                tag = f"AES-{len(key)*8}-{mode_name}-{name}"
                results[tag] = pt
                if b"%PDF" in pt or is_mostly_printable(pt[:200]) or b"flag" in pt:
                    print(f"[!] {tag} looks promising (contains printable/text or %PDF):")
                    print(pt[:1000])
            except Exception as e:
                print("AES try error", name, mode_name, e)
    return results

def try_rc4(secret_bytes: bytes, hint_bytes: bytes):
    print("[*] Trying RC4 (ARC4) with hint-derived keys...")
    candidates = []
    # raw hint, md5(hint), sha256[:16]
    candidates.append(hint_bytes)
    candidates.append(MD5.new(hint_bytes).digest())
    candidates.append(SHA256.new(hint_bytes).digest()[:16])
    results = {}
    for i, key in enumerate(candidates):
        try:
            cipher = ARC4.new(key)
            pt = cipher.decrypt(secret_bytes)
            tag = f"RC4-cand{i}"
            results[tag] = pt
            if is_mostly_printable(pt[:200]) or b"%PDF" in pt or b"flag" in pt:
                print(f"[!] {tag} looks promising:")
                print(pt[:500])
        except Exception as e:
            print("RC4 error", e)
    return results

# ---------- main flow ----------
def main(pdf_path: str):
    print("[*] Reading file and extracting hex fields...")
    fields = find_hex_fields(pdf_path)
    print("Found fields:", {k:(v[:80] + "..." if v and len(v)>80 else v) for k,v in fields.items()})
    if not fields["secret"]:
        print("No /Secret found in raw file. Exiting.")
        return

    secret_bytes = binascii.unhexlify(fields["secret"])
    hint_hex = fields["hint"]
    hint_bytes = binascii.unhexlify(hint_hex) if hint_hex else None

    # 1) quick human-readable inspect
    print("\n[*] Secret bytes length:", len(secret_bytes))
    if hint_bytes:
        print("[*] Hint bytes length:", len(hint_bytes), " =>", hint_bytes)
    else:
        print("[*] No hint bytes available.")

    # 2) Try XOR if hint exists
    if hint_bytes:
        xor_out = attempt_xor(secret_bytes, hint_bytes)
        if xor_out:
            # If output seems to contain password-like text, try to extract ascii tokens and test them
            cand_text = re.findall(rb"[ -~]{6,80}", xor_out)  # printable runs len>=6
            printed = set()
            for t in cand_text[:30]:
                s = t.decode('latin1', errors='ignore')
                if s not in printed:
                    printed.add(s)
                    print("[candidate token from XOR]:", s)
                    if try_pdf_decrypt(pdf_path, s):
                        return

    # 3) Try AES variants
    aes_results = try_aes_variants(secret_bytes, hint_bytes if hint_bytes else b"")
    # examine small printable candidates from AES outputs
    for tag, pt in aes_results.items():
        runs = re.findall(rb"[ -~]{6,200}", pt)
        for r in runs[:10]:
            s = r.decode('latin1', errors='ignore')
            print(f"[candidate token from {tag}]: {s}")
            if try_pdf_decrypt(pdf_path, s):
                return

    # 4) Try RC4
    rc4_results = try_rc4(secret_bytes, hint_bytes if hint_bytes else b"")
    for tag, pt in rc4_results.items():
        runs = re.findall(rb"[ -~]{6,200}", pt)
        for r in runs[:10]:
            s = r.decode('latin1', errors='ignore')
            print(f"[candidate token from {tag}]: {s}")
            if try_pdf_decrypt(pdf_path, s):
                return

    print("\n[*] None of the automatic attempts produced an obvious PDF password.")
    print("[*] Suggestions:")
    print("  - If you have a wordlist (e.g. rockyou.txt), run pdfcrack: pdfcrack share5.pdf -w rockyou.txt")
    print("  - Send me the full /Hint and /Secret hexes (if you're comfortable) and tôi sẽ thử trực tiếp.")
    print("  - If you suspect the hint encodes e.g. base64 or ascii, thử extract and show.")

main("share5.pdf")


[*] Reading file and extracting hex fields...
Found fields: {'hint': '18444671d7d45d1d', 'secret': 'b0d22125b97b2f258c570361446fbbe73984aeafd415048cfbe59cfaf6dce4ff7e0753b342b1ec16...', 'O': 'cdc580f5f2c092d8758e13223274b3b292a05262904480b37823a7e5aba91171', 'U': '4561b69a21d9e93bff57b40ea0eaf78500000000000000000000000000000000'}

[*] Secret bytes length: 114
[*] Hint bytes length: 8  => b'\x18DFq\xd7\xd4]\x1d'
[*] Trying repeating-key XOR with hint...
-> XOR result sample (first 200 bytes):
b'\xa8\x96gTn\xafr8\x94\x13E\x10\x93\xbb\xe6\xfa!\xc0\xe8\xde\x03\xc1Y\x91\xe3\xa1\xda\x8b!\x08\xb9\xe2fC\x15\xc2\x95e\xb1\x0b:,dq\xd4\xcbYG\xc8\x05nET\xa7\xa8\x14+\x98\x0c4\x8b\xbe4\xcb\x17)\x84\xf4\xf4\x8b\xb3G\x12\x19\xaeK5\xaf\xd1\x8b\x98\xf1\xd30\xcbv?\xe1\x1e\x9d\xd2\\s\xf5x\xde\xfc\xb9\xc3z\x15gW\xde\xae\xcc\x07\x97zl\x84\xeb\xda\xae'
[*] Trying AES variants (ECB/CBC) with keys derived from hint...
AES try error raw16 ECB Data must be aligned to block boundary in ECB mode
AES try error raw16

In [20]:
from PyPDF2 import PdfReader

reader = PdfReader("share5.pdf")
for pwd in ["Tlz='7&m", "tlz='7&m", "Tlz7&m", "7&mTlz=", "Tlz-7&m"]:
    res = reader.decrypt(pwd)
    print(pwd, "->", res)



Tlz='7&m -> PasswordType.NOT_DECRYPTED
tlz='7&m -> PasswordType.NOT_DECRYPTED
Tlz7&m -> PasswordType.NOT_DECRYPTED
7&mTlz= -> PasswordType.NOT_DECRYPTED
Tlz-7&m -> PasswordType.NOT_DECRYPTED


In [ ]:
#!/usr/bin/env python3
"""
exploit_pdf_run.py

Usage:
  python3 exploit_pdf_run.py input.txt
  cat input.txt | python3 exploit_pdf_run.py -
Notes:
  - Script will ignore arguments that start with '-' (useful in notebook/kernel env).
  - If '-' is passed or no filename given, script reads PDF text from stdin.
  - It reconstructs a PDF (adds header if missing), writes reconstructed.pdf,
    then tries to open with pypdf/PyPDF2 and attempt small password list.
"""
import sys, os

def read_input_data():
    # find first argv that does not start with '-' and is not the script name
    args = [a for a in sys.argv[1:] if not a.startswith('-')]
    if len(args) >= 1:
        fname = args[0]
        if fname == '-':
            data = sys.stdin.buffer.read()
            return data
        if not os.path.exists(fname):
            print(f"[-] File '{fname}' not found.")
            sys.exit(2)
        return open(fname, 'rb').read()
    else:
        # no positional arg: try stdin
        if not sys.stdin.isatty():
            return sys.stdin.buffer.read()
        else:
            print("Usage: python3 exploit_pdf_run.py input.txt  OR  cat input.txt | python3 exploit_pdf_run.py -")
            sys.exit(1)

def build_pdf_from_bytes(data, outfile='reconstructed.pdf'):
    if not data.startswith(b'%PDF-'):
        header = b"%PDF-1.7\n%\xE2\xE3\xCF\xD3\n\n"
        data = header + data
    if b'%%EOF' not in data:
        data = data + b"\nstartxref\n0\n%%EOF\n"
    open(outfile, 'wb').write(data)
    print('[+] Wrote reconstructed PDF to', outfile)
    return outfile

def try_pypdf_decrypt(pdf_path, passwords):
    try:
        from pypdf import PdfReader
    except Exception:
        try:
            from PyPDF2 import PdfFileReader as PdfReader
        except Exception as e:
            print('[-] pypdf/PyPDF2 not installed:', e)
            return False, None
    try:
        reader = PdfReader(pdf_path)
    except Exception as e:
        print('[-] Could not open PDF with pypdf/PyPDF2:', e)
        return False, None

    is_enc = False
    if hasattr(reader, 'is_encrypted'):
        is_enc = reader.is_encrypted
    elif hasattr(reader, 'isEncrypted'):
        is_enc = reader.isEncrypted

    if not is_enc:
        print('[+] PDF not encrypted according to pypdf.')
        return True, reader

    print('[*] PDF is encrypted. Trying passwords...')
    candidates = [''] + passwords
    for pwd in candidates:
        try:
            res = reader.decrypt(pwd)
            if res:
                print('[+] Decryption succeeded with password:', repr(pwd))
                return True, reader
        except Exception:
            try:
                reader.decrypt(pwd)
                print('[+] Decryption succeeded (alt) with password:', repr(pwd))
                return True, reader
            except Exception:
                pass
    print('[-] pypdf could not decrypt with provided passwords.')
    return False, None

def extract_and_save(reader, pdf_path, outdir='extracted_streams'):
    os.makedirs(outdir, exist_ok=True)
    try:
        pages = list(reader.pages)
    except Exception:
        pages = []
    extracted = 0
    for i, p in enumerate(pages):
        try:
            if hasattr(p, 'extract_text'):
                txt = p.extract_text()
                if txt and txt.strip():
                    open(os.path.join(outdir, f'page_{i+1}_text.txt'), 'w', encoding='utf-8', errors='ignore').write(txt)
                    extracted += 1
        except Exception:
            pass
    with open(os.path.join(outdir, 'full_pdf.bin'), 'wb') as f:
        f.write(open(pdf_path, 'rb').read())
    print(f'[+] Saved full_pdf.bin and extracted text from {extracted} pages to {outdir}')
    return extracted

if __name__ == '__main__':
    data = read_input_data()
    pdf_path = build_pdf_from_bytes(data, 'reconstructed.pdf')
    passwd_candidates = ['owner', 'password', '123456', 'admin', 'user', '']
    ok, reader = try_pypdf_decrypt(pdf_path, passwd_candidates)
    if ok:
        extract_and_save(reader, pdf_path)
    else:
        print('[-] Decryption not successful with small wordlist.')
        print('    Next: inspect object 54 /Encrypt (O, U, V, R) and use qpdf/pdf2john/john or implement PDF standard key derivation for more attempts.')


In [33]:
from PIL import Image
import pytesseract
img = Image.open("share7.png")
text = pytesseract.image_to_string(img)
print(text)


ABCDEFGHI<JSIKLUMNOPOQORSTUVUHWXYZ

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

4] <]@] 0] a]ea] u]eo] =|] 0] x2] ]22] =] [a] of a] m)e] >]>|s2] | >
|] <{-0]0/0]s0]s.[o] =|] 2]:[] =| =] 0]0|o1 eeu] | >] >|]
||| |@[o]a|s|u[o)=|-]>[)-/=|=[0]a/ole|u]e]>>|=)
=| | >[es[<[@[o/a/u/u[o|=/4/-) x) 4) =/=/0/a/ 01) 0) e)>[>
>| =|>|>|s[<]=]o[a|u/u]e]=[a|>]=| 3]2]=[0]a]ole/a/4]>)
>|>|=|=|>|-]<]=[o]a)u]u]e|e|4)>]2]]=|=]0]a[o]e) 0]q
| >|>[=|*|>|]<|@[0/0]]u.[o] =|] |x] [2] =] 0]a]o] «a
a) | >|>|=[=|=|=|<|@/ ofa] o|=|~]>|=[]=|=[0]a) ol «
e[o[e[>[>[2[x|>[e|<|@[o[afia)u|o|=|4/>/x))=)=/o]a) 01
of ee ale |>[>]=|>|>|]<]a]o]a)u)u]e|=|4[5]x] 3]2]=]0]0
a] of | a|e|>]>|=|=|>]-]<]=[o]a)u]u[e]=|4)>]x|]=/=| 0)
ofe| ofan] e|=]>[ [><] >[e0]<|ea) o/c) ea) us| 0) =| m)] x) a] =] =
=| =|0]e|of «| a/e-|>|>| =| >|» [ea c|ea)0/c9/ea) || =) v4) 2
[=| =[0]0) of«e{ale|>]>]=]>[>]]<]a]o]o/u)ule|=|4)a]s
|] =|=|0/a] ol«[a|4/>

In [43]:
def l2n(ch): return ord(ch)-65
def n2l(n): return chr((n%26)+65)

def decrypt_vigenere_letters(text, key):
    key_nums = [l2n(k) for k in key]
    out=[]
    ki=0
    for ch in text:
        if ch.isalpha():
            p = (l2n(ch.upper()) - key_nums[ki % len(key)]) % 26
            out.append(n2l(p))
            ki+=1
        else:
            out.append(ch)
    return ''.join(out)

cipher = "AM0ARHO DHA EASZOMOMOHDEDEMNN".replace('0','O')
start = 7  # vị trí letter-stream của 'DHA'

words3 = [
    "THE","AND","FOR","YOU","BUT","NOT","ALL","ANY","CAN","HAD",
    "HER","WAS","ONE","OUT","DAY","GET","HAS","HIM","HIS","NOW"
]

for word in words3:
    # tính key từ DHA -> word
    derived_key = word.upper()
    klen = len(derived_key)
    # xoay key theo pha
    rot = ['?']*klen
    for i,ch in enumerate(derived_key):
        pos = (start + i) % klen
        rot[pos] = ch
    rot_key = ''.join(rot)
    
    # giải ciphertext
    plaintext = decrypt_vigenere_letters(cipher, rot_key)
    print(f"TRY '{word}' -> rotated key: {rot_key}")
    print(plaintext)
    print("-"*60)


TRY 'THE' -> rotated key: ETH
WTHWYAK KAW LTOGHIVFKOWAKXIUG
------------------------------------------------------------
TRY 'AND' -> rotated key: DAN
XMBXRUL DUX ENPZBJOZLHQBDRJNA
------------------------------------------------------------
TRY 'FOR' -> rotated key: RFO
JHAJMTX YTJ ZMBUAVJYXCPNYQVIZ
------------------------------------------------------------
TRY 'YOU' -> rotated key: UYO
GOAGTTU FTG GMYBASQYUJPKFQSPZ
------------------------------------------------------------
TRY 'BUT' -> rotated key: TBU
HLUHQNV CNH DGZYUTNSVGJLCKTMT
------------------------------------------------------------
TRY 'NOT' -> rotated key: TNO
HZAHETV QTH RMZMATBYVUPLQQTAZ
------------------------------------------------------------
TRY 'ALL' -> rotated key: LAL
PMDPRWD DWP EPHZDBOBDHSTDTBNC
------------------------------------------------------------
TRY 'ANY' -> rotated key: YAN
CMBCRUQ DUC ENUZBOOZQHQGDRONA
------------------------------------------------------------
TRY 'CAN' -> rotated key: NCA
NK

In [1]:
print(len("AM0ARHO DHA EASZOMOMOHDEDEMNN"))

29
